# Bootstrapping Without Re-training

## Setup
Suppose we have a model $f: \mathcal X \to [0, 1]$ which predicts probabilities for some binary classificaiton problem with labels in $\mathcal Y = \{0, 1\}$, and we have some test set $D_\text{test} = \mathbf{X} \in \mathcal X^{N_{\text{data}}}, \mathbf{y} \in \mathcal Y^{N_{\text{data}}}$. We want to assess the efficacy of our classifier. So, we generate a bunch of probabilities $\mathbf p = f(\mathbf{X})$, $\mathbf p \in [0, 1]^{N_\text{data}}$.  We also have some _single_ threshold $t$ that we use for prediction, such that we form $\hat{y}_i = \begin{cases} 1 & \text{if } p_i > t \\ 0 & \text{otherwise.}\end{cases}$. Ultimately, now we have a set of _triples_ of probabilities and labels $(p_i, \hat{y}_i, y_i)$ for $0 \le i < N_\text{data}$.

We could just compute our evaluation on these straight-away. However, that would be silly -- we need variance! So instead, we decide (potentially erroneously) to do what we'll call in this notebook _bootstrapping_ (this may or may not be what is canonically meant by "bootstrapping"), by which I concretely mean that for $K$ iterations (where typically $K$ is large, e.g., $K >> 100$) we will choose a number of samples $N_\text{bootstrap}$ (which will _typically_ be $N_\text{data}$, but I'll allow the framework to be general so that we can disentangle their effects) from this set of triples randomly, independently, and with replacement, then compute some evaluation metric of interest on this $N_\text{bootstrap}$ subset. Over our $K$ samples, we can then compute confidence intervlas, means, variances, etc. With this setup, let's analyze a few concrete evaluation metrics we might care about.

## Dead Simple Evaluation Metrics

### Confusion Matrix Elements

Let's consider the confusion matrix elements _on the raw, original dataset_; in particular, the number of true positives, $TP$ (`TP`) which is the number of times $y_i = 1$ and $\hat{y}_i = 1$, the number of false positives, $FP$ (`FP`), which is the number of times $y_i = 0$ and $\hat{y}_i = 1$, the number of true negatives, $TN$ (`TN`), which is the number of times $y_i = 0$ and $\hat{y}_i = 0$, and the number of false negatives, $FN$ (`FN`), which is the number of times $y_i = 1$ and $\hat{y}_i = 0$. We'll add one more, which the number of "correct" points, $C$ (`C`), which is the number of points where $\hat{y}_i = y_i$. Mathematically:
  * $TP = \sum_{i=0}^{N_\text{data} - 1} \mathbb{1}_{y_i = 1} \cdot \mathbb{1}_{\hat{y}_i = 1}$
  * $FP = \sum_{i=0}^{N_\text{data} - 1} \mathbb{1}_{y_i = 0} \cdot \mathbb{1}_{\hat{y}_i = 1}$
  * $TN = \sum_{i=0}^{N_\text{data} - 1} \mathbb{1}_{y_i = 0} \cdot \mathbb{1}_{\hat{y}_i = 0}$
  * $FN = \sum_{i=0}^{N_\text{data} - 1} \mathbb{1}_{y_i = 1} \cdot \mathbb{1}_{\hat{y}_i = 0}$
  * $C = TP + TN = \sum_{i=0}^{N_\text{data}-1} \mathbb{1}_{y_i = \hat{y}_i}$

What happens to our estimates of these metrics under bootstrapping? Do we gain any insight into the variance we can expect for these numbers?

Well, note that any individual draw of our bootstrap process (e.g., any draw of a triple $(p^{(b)}, \hat{y}^{(b)}, y^{(b)})$ from our original set of triples during a bootstrap sample) is a draw from the empirical distribution of the dataset. Which means that as exactly $\frac{TP}{N_\text{data}}$ proportion of the samples have $\hat{y} = y = 1$, we see that this sample will output $\mathbb{1}_{y^{(b)} = 1} \cdot \mathbb{1}_{\hat{y}^{(b)} = 1} = 1$ according _precisely_ to a Bernoulli random variable with parameter given by $\frac{TP}{N_\text{data}}$. This holds as well for $FP$, $TN$, $FN$, and $C$ (all at their own proper rates). 

Thus, the induced number of true/false positives/negatives or the overall number of correct points under bootstrapping (for any given sample of the set of $K$ total bootstrap samples) therefore should follow a simple _binomial_ distribution with parameters (assume we are talking about true positives without loss of generality) of probability $p$ given by $\frac{TP}{N_\text{data}}$ and number of samples $N$ given by $N_\text{bootstrap}$. Thus, any derived property over the bootstrap samples should (provided $K$ is large) be determined simply by $TP$, $N_\text{data}$, and $N_\text{bootstrap}$. Thus the variance here has _minimal_ dependence on our model $f$. 

Note that this immediately offers insight into the measure "accuracy" as well, which is simply given by $\frac{C}{N}$ (where $N$ is the size of the dataset, either $N_\text{data}$ or $N_\text{bootstrap}$ depending).

### Compound Elements
Ok, so accuracy, $TP$, $FP$, $TN$, $FN$ and so on are all easy. But what about $TPR$, $FPR$, $TNR$, $FNR$? Recall that these rates are compounds:
  * $TPR = \frac{TP}{TP + FN} = \frac{TP}{\sum_{i=0}^{N_\text{data}-1} \mathbb{1}_{y_i = 1}}$
  
We can make this more analytical by considering breaking down our sampling problem. Let $PR = \frac{\sum_{i=0}^{N_\text{data}-1} \mathbb{1}_{y_i = 1}}{N_\text{data}}$ and $NR = 1- PR = \frac{\sum_{i=0}^{N_\text{data}-1} \mathbb{1}_{y_i = 0}}{N_\text{data}}$ be the positive and negative rate of our original dataset. With this parameter in mind, we can re-visit our original sampling procedure as, rather than sampling at random uniformly from the overall set of triples, we first choose wheter to sample a positive or negative example (with probability $PR$ v. $NR$), then within that choose to sample a predicted positive or predicted negative sample (with probability $TPR$ v. $FNR$ -OR- $FPR$ v. $TNR$).

Under this setting, it is clear that the probability our observed $TPR$ takes on value $\frac{m}{n}$ is given by: 
$$p(TPR_\text{obs} = \frac{m}{n} = p(P_\text{obs} = n \text{ AND } TP_\text{obs} = m)$$
But,
$$  p(P_\text{obs} = n \text{ AND } TP_\text{obs} = m) =  p(P_\text{obs} = n) p(TP_\text{obs} = m | P_\text{obs} = n)$$

The former probability in this product is given by a binomial random variable with parameters $PR$ and $N_\text{bootstrap}$. The second is given by $0$ if $m > n$ and a binomial random variable with parameters $TPR$ and $n$. Recall the probability of an $q, S$ binomial random variable taking on value $x$ is given by $P_{x} = {S \choose x} q^{x} (1-q)^{S-x}$.

Thus, the probability for us is:
$$p\left(TPR_\text{obs} = \frac{TP_\text{obs}}{P_\text{obs}}\right) = \left(
    {N_\text{bootstrap} \choose P_\text{obs}} PR^{P_\text{obs}}(1-PR)^{N_\text{bootstrap} - P_\text{obs}}
\right) \cdot \left(
    {P_\text{obs} \choose TP_\text{obs}} TPR^{TP_\text{obs}}(1-TPR)^{P_\text{obs} - TP_\text{obs}}
\right)$$

## AUROC

This one is... tougher. The formulation of interest to us here is that the AUROC is the probability that a randomly sampled positive example will be given an assigned probability that is higher than a randomly sampled negative example. The reason this is tricky is because when we take our bootstrap sample we sample at the level of individual data points, not pairs.

Note that the number of pairs of points $(i, j)$ such that $y_i = 1$ and $y_j = 0$ is precisely given by $P * N$ (or, in the bootstrap sampled, $P_\text{obs} * N_\text{obs}$). But, as everything is either $P$ or $N$, we can also rewrite this as $P (1 - P) = P - P**2$. 

This is our denominator in the AUROC calculation. The numerator is given by the number of pairs of points $(i, j)$ such that both $y_i = 1, y_j = 0$ _and_ that $p_i > p_j$. 

$$
p\left(\text{AUROC}_\text{obs} = \frac{R}{P_\text{obs}(1-P_\text{obs})}\right)
= p\left(\left.\text{AUROC}_\text{obs} = \frac{R}{P_\text{obs}(1-P_\text{obs})}\right| P_\text{obs} \right) \cdot p(P_\text{obs})
$$

$$ p\left(\text{AUROC}_\text{obs} = \frac{R}{P_\text{obs}(1-P_\text{obs})}\right)
= p\left(\left.\text{AUROC}_\text{obs} = \frac{R}{P_\text{obs}(1-P_\text{obs})}\right| P_\text{obs} \right) \cdot
\left(
    {N_\text{bootstrap} \choose P_\text{obs}} PR^{P_\text{obs}}(1-PR)^{N_\text{bootstrap} - P_\text{obs}}
\right)
$$